In [ ]:
# Install dependencies
%pip install --upgrade pip 
%pip install pandas pyarrow fastparquet

In [ ]:
import pandas as pd

# PARAMETERS
input_file = 'data/raw-data.csv'

In [ ]:
# TRANSFORM
print(f"Transformando datos de {input_file}...")
df = pd.read_csv(input_file)

df.columns = [col.strip().lower().replace(' ', '_').replace('.','_') for col in df.columns]

In [ ]:
df["adv_classify"] = df["adv_classify"].astype("category")
df["adv_tradetype"] = df["adv_tradetype"].astype("category")
df["adv_asset"]= df["adv_asset"].astype("category")
df["adv_fiatunit"]= df["adv_fiatunit"].astype("category")
df["adv_price"] = df["adv_price"].astype("float32")

In [ ]:
df = df.dropna(axis=1, how='all')
df.info()

In [ ]:
df.to_parquet('data/clean-data.parquet', index=False)
print("Transformación terminada.")